# Preprocessing

In [470]:
import pandas as pd

# read the data

df = pd.read_csv('train.csv')
df.describe()

,Characteristics.LotSizeSquareFeet,ImageData.c1c6.summary.bathroom,ImageData.c1c6.summary.exterior,ImageData.c1c6.summary.interior,ImageData.c1c6.summary.kitchen,ImageData.c1c6.summary.property,ImageData.q1q6.summary.bathroom,ImageData.q1q6.summary.exterior,ImageData.q1q6.summary.interior,ImageData.q1q6.summary.kitchen,...,Structure.BathroomsFull,Structure.BathroomsHalf,Structure.BedroomsTotal,Structure.BelowGradeFinishedArea,Structure.BelowGradeUnfinishedArea,Structure.FireplacesTotal,Structure.GarageSpaces,Structure.LivingArea,Structure.Rooms.RoomsTotal,Structure.YearBuilt
count,1.690000e+03,90744.000000,87789.000000,93597.000000,92320.000000,103055.000000,90708.000000,82565.000000,93589.000000,92292.000000,...,100063.000000,100049.000000,105024.000000,14235.000000,11674.000000,51216.000000,88621.000000,99509.000000,105061.000000,102256.000000
mean,5.949145e+04,3.116429,3.308723,3.211074,3.117166,3.182633,3.311895,3.499023,3.108145,3.209570,...,1.921669,0.441973,3.102167,701.495539,649.350608,0.941737,2.039585,1742.211448,7.174756,1969.155199
std,4.784652e+05,0.724160,0.676499,0.677497,0.743597,0.682011,0.508110,0.594215,0.540139,0.645833,...,0.864013,0.548588,1.235208,635.400209,577.616585,0.713055,8.542624,1144.743077,2.846424,53.661479
min,0.000000e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.743750e+03,2.800000,3.100000,2.900000,2.800000,2.900000,3.000000,3.100000,2.800000,2.800000,...,1.000000,0.000000,2.000000,0.000000,150.000000,1.000000,1.000000,1100.000000,5.000000,1952.000000
50%,8.712000e+03,3.200000,3.400000,3.300000,3.200000,3.200000,3.300000,3.500000,3.000000,3.200000,...,2.000000,0.000000,3.000000,696.000000,565.500000,1.000000,2.000000,1550.000000,7.000000,1973.000000
75%,2.452075e+04,3.600000,3.700000,3.600000,3.600000,3.600000,3.700000,3.900000,3.400000,3.800000,...,2.000000,1.000000,4.000000,1073.000000,1000.000000,1.000000,2.000000,2200.000000,8.000000,1998.000000
max,9.999999e+06,6.000000,6.000000,6.000000,6.000000,6.000000,5.100000,5.400000,5.100000,5.000000,...,75.000000,9.000000,18.000000,7782.000000,4896.000000,21.000000,999.000000,51400.000000,99.000000,2024.000000


The first step is to divide the data in Train and Validation

In [471]:
# divide train and validation data

from sklearn.model_selection import train_test_split

train, valid = train_test_split(df, test_size=0.2, random_state=42)

In [472]:
import pandas as pd
import numpy as np

def convert_to_missing(df):
    # Reemplazar todos los valores "missing" (NA, NaN, nan, None, '', 'none', 'na', etc.) con NaN
    missing_values = [None, 'NA', 'NaN', 'nan', '', 'none', 'na', 'Na', 'NULL']
    df = df.replace(missing_values, np.nan)
    return df

# Aplicar la función a los DataFrames de entrenamiento y validación
train = convert_to_missing(train)
valid = convert_to_missing(valid)

# Verificar que los valores faltantes hayan sido reemplazados
print(train.isna().sum())
print(valid.isna().sum())




Characteristics.LotFeatures               52372
Characteristics.LotSizeSquareFeet         84588
ImageData.c1c6.summary.bathroom           13386
ImageData.c1c6.summary.exterior           15699
ImageData.c1c6.summary.interior           11126
ImageData.c1c6.summary.kitchen            12142
ImageData.c1c6.summary.property            3485
ImageData.features_reso.results            2139
ImageData.q1q6.summary.bathroom           13415
ImageData.q1q6.summary.exterior           19907
ImageData.q1q6.summary.interior           11132
ImageData.q1q6.summary.kitchen            12165
ImageData.q1q6.summary.property            4498
ImageData.room_type_reso.results            453
ImageData.style.exterior.summary.label    19078
ImageData.style.stories.summary.label     19231
Listing.Dates.CloseDate                       0
Listing.ListingId                             0
Listing.Price.ClosePrice                      0
Location.Address.CensusBlock               5525
Location.Address.CensusTract            

In [473]:
# We create a table with the missing values and the percentage of missing values

missing_values = df.isnull().sum()
missing_values_percent = missing_values / len(df) * 100
missing_values_table = pd.concat([missing_values, missing_values_percent], axis=1)
missing_values_table.columns = ["Missing Values", "Percentage"]
missing_values_table = missing_values_table[missing_values_table["Missing Values"] > 0]
missing_values_table = missing_values_table.sort_values(by="Missing Values", ascending=False)
print(missing_values_table)


                                        Missing Values  Percentage
Location.Address.StreetDirectionSuffix          106796   99.403371
Characteristics.LotSizeSquareFeet               105747   98.426985
Location.Address.PostalCodePlus4                104166   96.955425
UnitTypes.UnitTypeType                          102461   95.368448
Tax.Zoning                                       99939   93.021026
Structure.BelowGradeUnfinishedArea               95763   89.134097
Structure.ParkingFeatures                        93880   87.381442
Structure.BelowGradeFinishedArea                 93202   86.750375
Location.Address.UnitNumber                      82894   77.155915
Location.Area.SubdivisionName                    70896   65.988440
Characteristics.LotFeatures                      65417   60.888707
Location.Address.StreetDirectionPrefix           58743   54.676694
Structure.FireplacesTotal                        56221   52.329272
ImageData.q1q6.summary.exterior                  24872   23.15

In [474]:
# Mirar los valores unicos de street direction sufix
print(train['Location.Address.StreetDirectionSuffix'].unique())

# Mirar los valores unicos de street direction prefix
print(train['Location.Address.StreetDirectionPrefix'].unique())

# Unir las columnas anteriores en una que sea street direction
train['Location.Address.StreetDirection'] = train['Location.Address.StreetDirectionPrefix'].fillna('') + train['Location.Address.StreetDirectionSuffix'].fillna('')
valid['Location.Address.StreetDirection'] = valid['Location.Address.StreetDirectionPrefix'].fillna('') + valid['Location.Address.StreetDirectionSuffix'].fillna('')

# Eliminar las columnas que ya no necesitamos
train.drop(columns=['Location.Address.StreetDirectionPrefix', 'Location.Address.StreetDirectionSuffix'], inplace=True)
valid.drop(columns=['Location.Address.StreetDirectionPrefix', 'Location.Address.StreetDirectionSuffix'], inplace=True)

#Los NA en el formato correcto
train['Location.Address.StreetDirection'] = train['Location.Address.StreetDirection'].replace('', None)
valid['Location.Address.StreetDirection'] = valid['Location.Address.StreetDirection'].replace('', None)





[nan 'e' 'n' 'w' 's' 'se' 'sw' 'ne' 'nw']
[nan 'e' 's' 'n' 'w' 'sw' 'ne' 'nw' 'se']


In [475]:
# Mirar los missing values de la nueva variable
print(train['Location.Address.StreetDirection'].isnull().sum())
print(valid['Location.Address.StreetDirection'].isnull().sum())

#calcular los porcentages de missings
print(train['Location.Address.StreetDirection'].isnull().sum()/len(train))
print(valid['Location.Address.StreetDirection'].isnull().sum()/len(valid))

46632
11692
0.5425543054602148
0.5441176470588235


In [476]:
#Eliminem les columnes 
train.drop(columns=['Characteristics.LotSizeSquareFeet', 'Location.Address.PostalCodePlus4'], inplace=True)

In [477]:
# Eliminar las columnas que ya no necesitamos
train.drop(columns=['UnitTypes.UnitTypeType', 'Tax.Zoning'], inplace=True)
valid.drop(columns=['UnitTypes.UnitTypeType', 'Tax.Zoning'], inplace=True)

# Mirar los missing values de la nueva variable
missing_values = train.isnull().sum()
missing_values_percent = missing_values / len(train) * 100
missing_values_table = pd.concat([missing_values, missing_values_percent], axis=1)
missing_values_table.columns = ["Missing Values", "Percentage"]
missing_values_table = missing_values_table[missing_values_table["Missing Values"] > 0]
missing_values_table = missing_values_table.sort_values(by="Missing Values", ascending=False)
print(missing_values_table)


                                        Missing Values  Percentage
Structure.BelowGradeUnfinishedArea               76661   89.193592
Structure.ParkingFeatures                        75135   87.418120
Structure.BelowGradeFinishedArea                 74587   86.780533
Location.Address.UnitNumber                      66274   77.108518
Location.Area.SubdivisionName                    56740   66.015893
Characteristics.LotFeatures                      52372   60.933810
Location.Address.StreetDirection                 46632   54.255431
Structure.FireplacesTotal                        45036   52.398515
ImageData.q1q6.summary.exterior                  19907   23.161410
ImageData.style.stories.summary.label            19231   22.374897
ImageData.style.exterior.summary.label           19078   22.196884
ImageData.c1c6.summary.exterior                  15699   18.265483
Structure.GarageSpaces                           14994   17.445229
ImageData.q1q6.summary.bathroom                  13415   15.60

In [478]:
#Structure.ParkingFeatures si tiene resultado 1 si no 0
train['Structure.ParkingFeatures'] = train['Structure.ParkingFeatures'].notnull().astype(int)
valid['Structure.ParkingFeatures'] = valid['Structure.ParkingFeatures'].notnull().astype(int)

# Mirar los missing values de la nueva variable
missing_values = train.isnull().sum()
missing_values_percent = missing_values / len(train) * 100
missing_values_table = pd.concat([missing_values, missing_values_percent], axis=1)
missing_values_table.columns = ["Missing Values", "Percentage"]
missing_values_table = missing_values_table[missing_values_table["Missing Values"] > 0]
missing_values_table = missing_values_table.sort_values(by="Missing Values", ascending=False)
print(missing_values_table)



                                        Missing Values  Percentage
Structure.BelowGradeUnfinishedArea               76661   89.193592
Structure.BelowGradeFinishedArea                 74587   86.780533
Location.Address.UnitNumber                      66274   77.108518
Location.Area.SubdivisionName                    56740   66.015893
Characteristics.LotFeatures                      52372   60.933810
Location.Address.StreetDirection                 46632   54.255431
Structure.FireplacesTotal                        45036   52.398515
ImageData.q1q6.summary.exterior                  19907   23.161410
ImageData.style.stories.summary.label            19231   22.374897
ImageData.style.exterior.summary.label           19078   22.196884
ImageData.c1c6.summary.exterior                  15699   18.265483
Structure.GarageSpaces                           14994   17.445229
ImageData.q1q6.summary.bathroom                  13415   15.608093
ImageData.c1c6.summary.bathroom                  13386   15.57

In [479]:
# Unknown a Location.Area.SubdivisionName   
train['Location.Area.SubdivisionName'] = train['Location.Area.SubdivisionName'].replace('', None)
valid['Location.Area.SubdivisionName'] = valid['Location.Area.SubdivisionName'].replace('', None)

# Mirar los missing values de la nueva variable
missing_values = train.isnull().sum()
missing_values_percent = missing_values / len(train) * 100
missing_values_table = pd.concat([missing_values, missing_values_percent], axis=1)
missing_values_table.columns = ["Missing Values", "Percentage"]
missing_values_table = missing_values_table[missing_values_table["Missing Values"] > 0]
missing_values_table = missing_values_table.sort_values(by="Missing Values", ascending=False)
print(missing_values_table)


                                        Missing Values  Percentage
Structure.BelowGradeUnfinishedArea               76661   89.193592
Structure.BelowGradeFinishedArea                 74587   86.780533
Location.Address.UnitNumber                      66274   77.108518
Location.Area.SubdivisionName                    56740   66.015893
Characteristics.LotFeatures                      52372   60.933810
Location.Address.StreetDirection                 46632   54.255431
Structure.FireplacesTotal                        45036   52.398515
ImageData.q1q6.summary.exterior                  19907   23.161410
ImageData.style.stories.summary.label            19231   22.374897
ImageData.style.exterior.summary.label           19078   22.196884
ImageData.c1c6.summary.exterior                  15699   18.265483
Structure.GarageSpaces                           14994   17.445229
ImageData.q1q6.summary.bathroom                  13415   15.608093
ImageData.c1c6.summary.bathroom                  13386   15.57

In [480]:
#Eliminar la Location.Address.UnitNumber
train.drop(columns=['Location.Address.UnitNumber'], inplace=True)
valid.drop(columns=['Location.Address.UnitNumber'], inplace=True)


In [481]:
#Mirar los valores unicos de la variable Location.Address.StateOrProvince
print(train['Location.Address.StateOrProvince'].unique())
train.drop(columns=['Location.Address.StateOrProvince'], inplace=True)

['il']


In [ ]:
#IMPUTAR Structure.LivingArea 

In [505]:
missing_values = train.isnull().sum()
missing_values_percent = missing_values / len(train) * 100
missing_values_table = pd.concat([missing_values, missing_values_percent], axis=1)
missing_values_table.columns = ["Missing Values", "Percentage"]
missing_values_table = missing_values_table[missing_values_table["Missing Values"] > 0]
missing_values_table = missing_values_table.sort_values(by="Missing Values", ascending=False)
print(missing_values_table)

                                        Missing Values  Percentage
Characteristics.LotFeatures                      52327   60.916182
ImageData.style.stories.summary.label            19207   22.359721
ImageData.style.exterior.summary.label           19054   22.181607
ImageData.c1c6.summary.exterior                  15674   18.246799
Structure.GarageSpaces                           14964   17.420256
ImageData.q1q6.summary.bathroom                  13387   15.584400
ImageData.c1c6.summary.bathroom                  13358   15.550640
ImageData.q1q6.summary.kitchen                   12138   14.130384
ImageData.c1c6.summary.kitchen                   12115   14.103609
ImageData.q1q6.summary.interior                  11105   12.927823
ImageData.c1c6.summary.interior                  11099   12.920838
Structure.LivingArea                              6310    7.345751
Structure.Cooling                                 4719    5.493597
ImageData.q1q6.summary.property                   4475    5.20

In [507]:
!pip install impyute

In [ ]:
from impyute.imputation.cs import mice

columns_to_impute = [
    "ImageData.q1q6.summary.bathroom",
    "ImageData.c1c6.summary.bathroom",
    "ImageData.q1q6.summary.kitchen",
    "ImageData.c1c6.summary.kitchen",
    "ImageData.q1q6.summary.interior",
    "ImageData.c1c6.summary.interior",
    "ImageData.q1q6.summary.exterior",
    "ImageData.c1c6.summary.exterior",
    "ImageData.q1q6.summary.property",
    "ImageData.c1c6.summary.property"
]

imputed_data = mice(train[columns_to_impute].to_numpy())

# Reemplazar las columnas originales con los valores imputados
train[columns_to_impute] = imputed_data

In [506]:
import numpy as np

# Crear la columna combinada con la media de los valores, pero si uno es NA, rellenar ese valor con 

# Imputar valores nulos
train['ImageData.c1c6.summary.property'] = train['ImageData.c1c6.summary.property'].fillna(train['ImageData.c1c6.summary.property'].mean())
train['ImageData.q1q6.summary.property'] = train['ImageData.q1q6.summary.property'].fillna(train['ImageData.q1q6.summary.property'].mean())

# Crear la columna combinada (media de las dos columnas después de imputar)
train['ImageData.summary.exterior'] = (train['ImageData.c1c6.summary.property'] + train['ImageData.q1q6.summary.property']) / 2

# Eliminar las columnas originales (opcional)
train.drop(columns=['ImageData.c1c6.summary.property', 'ImageData.q1q6.summary.property'], inplace=True)



In [460]:
# juntamos ImageData.c1c6.summary.exterior, ImageData.q1q6.summary.exterior con la media de los valores no nulos
train['ImageData.c1c6.summary.exterior'] = train['ImageData.c1c6.summary.exterior'].astype(float)
train['ImageData.q1q6.summary.exterior'] = train['ImageData.q1q6.summary.exterior'].astype(float)
train['ImageData.summary.exterior'] = (train['ImageData.c1c6.summary.exterior'] + train['ImageData.q1q6.summary.exterior']) / 2

# Eliminar las columnas que ya no necesitamos
train.drop(columns=['ImageData.c1c6.summary.exterior', 'ImageData.q1q6.summary.exterior'], inplace=True)
valid.drop(columns=['ImageData.c1c6.summary.exterior', 'ImageData.q1q6.summary.exterior'], inplace=True)




In [466]:
# juntamos ImageData.c1c6.summary.exterior, ImageData.q1q6.summary.exterior con la media de los valores no nulos
train['ImageData.q1q6.summary.interior'] = train['ImageData.q1q6.summary.interior'].astype(float)
train['ImageData.c1c6.summary.interior'] = train['ImageData.c1c6.summary.interior'].astype(float)
train['ImageData.summary.interior'] = (train['ImageData.c1c6.summary.interior'] + train['ImageData.q1q6.summary.interior']) / 2

# Eliminar las columnas que ya no necesitamos
train.drop(columns=['ImageData.c1c6.summary.interior', 'ImageData.q1q6.summary.interior'], inplace=True)
valid.drop(columns=['ImageData.c1c6.summary.interior', 'ImageData.q1q6.summary.interior'], inplace=True)

In [462]:
train['ImageData.q1q6.summary.bathroom'] = train['ImageData.q1q6.summary.bathroom'].astype(float)
train['ImageData.c1c6.summary.bathroom'] = train['ImageData.c1c6.summary.bathroom'].astype(float)
train['ImageData.summary.bathroom'] = (train['ImageData.q1q6.summary.bathroom'] + train['ImageData.c1c6.summary.bathroom']) / 2

# Eliminar las columnas que ya no necesitamos
train.drop(columns=['ImageData.q1q6.summary.bathroom', 'ImageData.c1c6.summary.bathroom'], inplace=True)


In [464]:
train['ImageData.q1q6.summary.kitchen'] = train['ImageData.q1q6.summary.kitchen'].astype(float)
train['ImageData.c1c6.summary.kitchen'] = train['ImageData.c1c6.summary.kitchen'].astype(float)
train['ImageData.summary.kitchen'] = (train['ImageData.q1q6.summary.kitchen'] + train['ImageData.c1c6.summary.kitchen']) / 2

# Eliminar las columnas que ya no necesitamos
train.drop(columns=['ImageData.q1q6.summary.kitchen', 'ImageData.c1c6.summary.kitchen'], inplace=True)

In [ ]:
# Mirem dels missings que tenim qui

In [483]:
# Reemplazamos Location.Address.CountyOrParish y Location.Address.StreetName de missing a categoria Unknown
train['Location.Address.CountyOrParish'] = train['Location.Address.CountyOrParish'].replace(np.nan, 'Unknown')
train['Location.Address.StreetName'] = train['Location.Address.StreetName'].replace(np.nan, 'Unknown')

valid['Location.Address.CountyOrParish'] = valid['Location.Address.CountyOrParish'].replace(np.nan, 'Unknown')
valid['Location.Address.StreetName'] = valid['Location.Address.StreetName'].replace(np.nan, 'Unknown')


In [ ]:
# Structure.Heating dividir llista i veure correlacions, si no té 0
# Charactersitics.Lot.Features = dividir llista
# ImageData.features_reso.results = dividir llista

In [293]:
!pip install geopy

In [484]:
#girem les columnes de latitude i longitude
# Intercambiar las columnas de latitud y longitud
train['Location.GIS.Latitude'], train['Location.GIS.Longitude'] = train['Location.GIS.Longitude'], train['Location.GIS.Latitude']
valid['Location.GIS.Latitude'], valid['Location.GIS.Longitude'] = valid['Location.GIS.Longitude'], valid['Location.GIS.Latitude']



In [485]:
# Location.Address.StreetSuffix los missings por Unknown
train['Location.Address.StreetSuffix'] = train['Location.Address.StreetSuffix'].replace('', 'Unknown')
valid['Location.Address.StreetSuffix'] = valid['Location.Address.StreetSuffix'].replace('', 'Unknown')


In [486]:
#Location.Address.CountyOrParish los missings por Unknown
train['Location.Address.CountyOrParish'] = train['Location.Address.CountyOrParish'].replace('', 'Unknown')
valid['Location.Address.CountyOrParish'] = valid['Location.Address.CountyOrParish'].replace('', 'Unknown')


In [487]:
#Structure.NewConstructionYN miramos la edad de la casa y si es nueva o no
train['Structure.NewConstructionYN'] = (train['Structure.YearBuilt'] <= 1).astype(int)

In [488]:
# Borrar street number
train.drop(columns=['Location.Address.StreetNumber'], inplace=True)
valid.drop(columns=['Location.Address.StreetNumber'], inplace=True)

#Street Name missings a Unknown
train['Location.Address.StreetName'] = train['Location.Address.StreetName'].replace('', 'Unknown')
valid['Location.Address.StreetName'] = valid['Location.Address.StreetName'].replace('', 'Unknown')


In [489]:
import ast

# Función para convertir cadenas que parecen listas en listas reales y tomar el primer valor si es una lista
def convert_to_first_value(x):
    if isinstance(x, str):
        try:
            # Intentar convertir la cadena a una lista real
            x = ast.literal_eval(x)
        except (ValueError, SyntaxError):
            pass  # Si no se puede convertir, dejamos el valor tal cual
    if isinstance(x, list) and len(x) > 0:
        return x[0]  # Si es una lista, devolver el primer valor
    return x  # Si no es una lista, devolver el valor tal cual

# Aplicar la función a la columna 'Location.School.HighSchoolDistrict'
train['Location.School.HighSchoolDistrict'] = train['Location.School.HighSchoolDistrict'].apply(convert_to_first_value)

# Verificar los valores únicos después de la modificación
print(train['Location.School.HighSchoolDistrict'].unique())



[200 '158' 308 '154' 155 4 299 227 219 205 60 nan 125 202 203 '299' '60'
 '207' 99 101 12 '202' 225 120 '87' 307 '140' 68 204 207 430 '5' 100 233
 118 '208' '46' 211 '424' 303 302 217 229 215 25 201 '210' 129 '215' 220
 '214' 158 '155' '120' 212 '116' 5 '61' '129' 1 '127' 46 170 90 122 427
 '101' 117 137 209 111 '99' 228 131 214 '225' 156 230 '200' 127 88 '401'
 126 432 '205' '125' '212' 300 '201' 208 '204' 87 16 502 61 '201u' '4' '1'
 '126' 121 210 124 '218' '86' 94 130 218 'chris' 40 '8' 206 223 301 73 116
 '115' '3' 86 95 128 '200u' 113 500 143.5 '111' 145 424 11 '170' '255u'
 '128' '88' 10 '117' '228' 2 'other' '100' '209' 187 '230' '303' 426 '701'
 '301' 231 108 '229' '233' '211' '219' '427' 193 18 154 '203' '95' '275'
 '307' '193' 140 '207u' '308' '15' '220' 305 428 304 323 157 '365u' '227'
 15 '300' '122' 21 '108' 3 '304' 7 '96' '217' '6' 221 54 160 '6-j' '302'
 '305' '131' '2' 401 '113' '18' '118' 115 74 6 314 '137' '21' '121' '156'
 165 425 8 '206' 9 '124' 50 152 429 '161' '53

In [490]:
# Quitamos los warnings
import warnings
warnings.filterwarnings('ignore')



In [491]:
import pandas as pd
from geopy.distance import geodesic
from collections import defaultdict
import numpy as np

# Función para calcular la distancia entre dos coordenadas (latitud, longitud)
def calculate_distance(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).km

# Precalcular las distancias entre las casas dentro de cada código postal y almacenar los nombres de las escuelas
def precalculate_distances(train_df):
    print('precalculate_distances')
    distances = defaultdict(list)
    
    # Iteramos sobre las casas para organizar por código postal
    for idx, house in train_df.iterrows():
        house_lat = house['Location.GIS.Latitude']
        house_lon = house['Location.GIS.Longitude']
        postal_code = house['Location.Address.CensusBlock']
        school_name = house['Location.School.HighSchoolDistrict']
        
        # Validar que los valores necesarios existan
        if not pd.isna(house_lat) and not pd.isna(house_lon) and not pd.isna(postal_code):
            distances[postal_code].append((house_lat, house_lon, school_name))  # Guardamos la escuela
    
    return distances

# Función para obtener la escuela más cercana
def get_nearest_school(lat, lon, postal_code, distances):
    # Buscar la escuela más cercana dentro del mismo código postal
    if postal_code not in distances:
        return 'unknown'  # Si no hay casas en el código postal, devolvemos 'unknown'
    
    # Ordenar las casas dentro del mismo código postal por distancia
    distances_sorted = sorted(distances[postal_code], key=lambda x: calculate_distance(lat, lon, x[0], x[1]))
    
    # Buscar el primer nombre de escuela no nulo
    for _, _, school_name in distances_sorted:
        if school_name:  # Si encontramos una escuela válida
            return school_name
    
    return 'unknown'  # Si no se encontró ninguna escuela válida

# Función para llenar los valores faltantes de 'Location.School.HighSchoolDistrict'
def fill_missing_districts(df, distances):
    print('fill_missing_districts')

    # Aplicar la función para buscar escuelas cercanas solo en filas donde falte el distrito escolar
    missing_idx = df['Location.School.HighSchoolDistrict'].isna()
    df.loc[missing_idx, 'Location.School.HighSchoolDistrict'] = df[missing_idx].apply(
        lambda row: get_nearest_school(row['Location.GIS.Latitude'], 
                                       row['Location.GIS.Longitude'], 
                                       row['Location.Address.CensusBlock'], 
                                       distances),
        axis=1
    )

    # Reemplazar cualquier valor faltante restante con "unknown"
    df['Location.School.HighSchoolDistrict'] = df['Location.School.HighSchoolDistrict'].fillna('unknown')
    return df

# Precalcular las distancias para el DataFrame de entrenamiento
distances = precalculate_distances(train)

# Llenar valores faltantes en 'train' y 'valid'
train = fill_missing_districts(train, distances)
valid = fill_missing_districts(valid, distances)

# Verificar que no haya valores faltantes
assert train['Location.School.HighSchoolDistrict'].isna().sum() == 0, "Quedan valores faltantes en train"
assert valid['Location.School.HighSchoolDistrict'].isna().sum() == 0, "Quedan valores faltantes en valid"

print("Todos los valores faltantes han sido rellenados.")


precalculate_distances
fill_missing_districts
fill_missing_districts
Todos los valores faltantes han sido rellenados.


In [492]:
# Structure.BathroomsHalf, Structure.BathroomsFull los NA los ponemos a 0
train['Structure.BathroomsHalf'] = train['Structure.BathroomsHalf'].fillna(0)
valid['Structure.BathroomsHalf'] = valid['Structure.BathroomsHalf'].fillna(0)

train['Structure.BathroomsFull'] = train['Structure.BathroomsFull'].fillna(0)
valid['Structure.BathroomsFull'] = valid['Structure.BathroomsFull'].fillna(0)


In [493]:
#Structure.Rooms.RoomsTotal los NA los ponemos a 0
train['Structure.Rooms.RoomsTotal'] = train['Structure.Rooms.RoomsTotal'].fillna(0)
valid['Structure.Rooms.RoomsTotal'] = valid['Structure.Rooms.RoomsTotal'].fillna(0)

In [494]:
# Structure.BedroomsTotal  los NA los ponemos a 0
train['Structure.BedroomsTotal'] = train['Structure.BedroomsTotal'].fillna(0)
valid['Structure.BedroomsTotal'] = valid['Structure.BedroomsTotal'].fillna(0)                      


In [495]:
# Structure.FireplacesTotal los NA a 0
train['Structure.FireplacesTotal'] = train['Structure.FireplacesTotal'].fillna(0)
valid['Structure.FireplacesTotal'] = valid['Structure.FireplacesTotal'].fillna(0)




In [496]:
# Pels missing Location.GIS.Longitude,Location.GIS.Latitude calculem el valor mitja segons Location.Address.PostalCode
train['Location.GIS.Longitude'] = train['Location.GIS.Longitude'].fillna(train.groupby('Location.Address.PostalCode')['Location.GIS.Longitude'].transform('mean'))
train['Location.GIS.Latitude'] = train['Location.GIS.Latitude'].fillna(train.groupby('Location.Address.PostalCode')['Location.GIS.Latitude'].transform('mean'))

In [497]:
def fill_mode(group):
    mode_value = group.mode()
    if not mode_value.empty:
        return mode_value.iloc[0]
    return group.iloc[0]  

train['Location.Address.CensusBlock'] = train.groupby('Location.Address.PostalCode')['Location.Address.CensusBlock'].transform(fill_mode)
train['Location.Address.CensusTract'] = train.groupby('Location.Address.PostalCode')['Location.Address.CensusTract'].transform(fill_mode)


In [498]:
#borrar las rows de Location.Address.CensusBlock, Location.Address.CensusTract que tienen missing

train = train.dropna(subset=['Location.Address.CensusBlock', 'Location.Address.CensusTract'])


In [499]:
# ImageData.q1q6.summary.exterior Mitjana segons el census block group
train['ImageData.q1q6.summary.exterior'] = train['ImageData.q1q6.summary.exterior'].fillna(train.groupby('Location.Address.CensusBlock')['ImageData.q1q6.summary.exterior'].transform('mean'))
valid['ImageData.q1q6.summary.exterior'] = valid['ImageData.q1q6.summary.exterior'].fillna(valid.groupby('Location.Address.CensusBlock')['ImageData.q1q6.summary.exterior'].transform('mean'))


In [500]:
# Structure.BelowGradeUnfinishedArea, Structure.BelowGradeFinishedArea los NA a 0
train['Structure.BelowGradeUnfinishedArea'] = train['Structure.BelowGradeUnfinishedArea'].fillna(0)
train['Structure.BelowGradeFinishedArea'] = train['Structure.BelowGradeFinishedArea'].fillna(0)
valid['Structure.BelowGradeUnfinishedArea'] = valid['Structure.BelowGradeUnfinishedArea'].fillna(0)
valid['Structure.BelowGradeFinishedArea'] = valid['Structure.BelowGradeFinishedArea'].fillna(0)



In [501]:
# Location.Area.SubdivisionName los NA a Unknown (substituir en un futur) 
train['Location.Area.SubdivisionName'] = train['Location.Area.SubdivisionName'].fillna('Unknown')
valid['Location.Area.SubdivisionName'] = valid['Location.Area.SubdivisionName'].fillna('Unknown')



In [502]:
train['Location.Address.StreetDirection'] = train['Location.Address.StreetDirection'].fillna('Unknown')
valid['Location.Address.StreetDirection'] = valid['Location.Address.StreetDirection'].fillna('Unknown')

In [503]:
#Structure.FireplacesTotal   los NA a 0
train['Structure.FireplacesTotal'] = train['Structure.FireplacesTotal'].fillna(0)
valid['Structure.FireplacesTotal'] = valid['Structure.FireplacesTotal'].fillna(0)


In [504]:
#En el year built calcular la edad de la casa (2024 - year built)
train['Structure.YearBuilt'] = 2024 - train['Structure.YearBuilt']
valid['Structure.YearBuilt'] = 2024 - valid['Structure.YearBuilt']

#Structure.YearBuilt los NA a la mediana
train['Structure.YearBuilt'] = train['Structure.YearBuilt'].fillna(train['Structure.YearBuilt'].median())
valid['Structure.YearBuilt'] = valid['Structure.YearBuilt'].fillna(valid['Structure.YearBuilt'].median())





In [399]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from scipy.stats import spearmanr
import ast

train['ImageData.features_reso.results'] = train['ImageData.features_reso.results'].apply(
    lambda x: x if isinstance(x, list) else [] if pd.isnull(x) else ast.literal_eval(x)
)

mlb = MultiLabelBinarizer()
binary_features = mlb.fit_transform(train['ImageData.features_reso.results'])
binary_df = pd.DataFrame(binary_features, columns=mlb.classes_, index=train.index)

# Concatenar con el DataFrame original
df = pd.concat([train, binary_df], axis=1)

new_columns = list(mlb.classes_)
print(new_columns)

print(len(new_columns))

KeyboardInterrupt: 

                                Appliances.BarFridge  \
Appliances.BarFridge                        1.000000   
Appliances.BuiltInRefrigerator              0.004493   
Appliances.Cooktop                          0.010374   
Appliances.Dishwasher                       0.004682   
Appliances.DoubleOven                       0.001861   
...                                              ...   
WaterfrontFeatures.Lake                    -0.000635   
WaterfrontFeatures.Pond                    -0.001676   
WaterfrontFeatures.Waterfront              -0.000099   
WindowFeatures.Skylights                    0.001375   
Listing.Price.ClosePrice                    0.010817   

                                Appliances.BuiltInRefrigerator  \
Appliances.BarFridge                                  0.004493   
Appliances.BuiltInRefrigerator                        1.000000   
Appliances.Cooktop                                    0.262453   
Appliances.Dishwasher                                 0.039785 